In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages  org.apache.spark:spark-avro_2.12:3.4.1,io.delta:delta-core_2.12:2.4.0 pyspark-shell'
spark = SparkSession.builder.master("local[*]").appName("Spark_ML") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .enableHiveSupport()\
        .getOrCreate()

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

your 131072x1 screen size is bogus. expect trouble
23/07/05 00:23:33 WARN Utils: Your hostname, DESKTOP-JKDOQO9 resolves to a loopback address: 127.0.1.1; using 172.26.19.142 instead (on interface eth0)
23/07/05 00:23:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/josealcocer27/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/josealcocer27/.ivy2/cache
The jars for the packages stored in: /home/josealcocer27/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c5ecbbc8-3c26-4e29-900f-63e4be918a77;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.4.1 in central
	found org.tukaani#xz;1.9 in central
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 293ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.spark#spark-avro_2.12;3.4.1 from central in [default]
	org.tukaani#xz;1.9 from central in [default]
	--------------------------------------------

In [10]:
df = spark.read.format('delta')\
    .load('delta/saber11')

In [11]:
df.count()

23/07/05 00:26:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


10361

In [12]:
df.printSchema()

root
 |-- lenguaje: string (nullable = true)
 |-- matematicas: string (nullable = true)
 |-- sociales: string (nullable = true)
 |-- Id: integer (nullable = true)
 |-- filosofia: string (nullable = true)
 |-- biologia: string (nullable = true)
 |-- quimica: string (nullable = true)
 |-- fisica: string (nullable = true)
 |-- nivel_ingles: string (nullable = true)
 |-- sisben: integer (nullable = true)
 |-- estrato: integer (nullable = true)
 |-- genero: string (nullable = true)
 |-- puntaje_saber11: string (nullable = true)



In [13]:
for column in df.columns:
    if column != 'Id':
        print(f'\nUnique values at column {column}\n')
        df.select(column).distinct().show()


Unique values at column lenguaje



+--------+
|lenguaje|
+--------+
|    bajo|
|   medio|
|superior|
|    alto|
+--------+


Unique values at column matematicas

+-----------+
|matematicas|
+-----------+
|       bajo|
|      medio|
|   superior|
|       alto|
+-----------+


Unique values at column sociales

+--------+
|sociales|
+--------+
|    bajo|
|   medio|
|superior|
|    alto|
+--------+


Unique values at column filosofia

+---------+
|filosofia|
+---------+
|     bajo|
|    medio|
| superior|
|     alto|
+---------+


Unique values at column biologia

+--------+
|biologia|
+--------+
|    bajo|
|   medio|
|superior|
|    alto|
+--------+


Unique values at column quimica

+--------+
| quimica|
+--------+
|    bajo|
|   medio|
|superior|
|    alto|
+--------+


Unique values at column fisica

+--------+
|  fisica|
+--------+
|    bajo|
|superior|
|   medio|
|    alto|
+--------+


Unique values at column nivel_ingles

+------------+
|nivel_ingles|
+------------+
|          A2|
|          A-|
|          B1|
|    

In [14]:
df = df.drop('id')
df = df.dropna()